In [1]:
import numpy as np
import pandas as pd
import operator
import itertools
from itertools import combinations, chain

In [18]:
data= pd.read_csv(r"C:\Users\diyas\Documents\Recommendation_system\MasterDatasetTemp_Preprocessed_Anonymized_small.csv") 

In [19]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'USN', '10thPercentage', '10thMarks',
       '12thPercentage', 'Branch', 'CGPA', 'CourseName', 'CourseGrade',
       'CompanyName_New', 'CTC', 'TierLevel', 'CoCurricularActivities',
       'EmploymentType', 'EventsParticipated', 'GeneralSkills', 'Languages',
       'MinorAttended', 'NoofInternships', 'InternshipProjectDomain',
       'InternshipCompany', 'NoofProjects', 'ProjectDetailDomain',
       'ProgLanguages', 'Publication', 'ResearchDomain', 'ScholarshipsKey',
       'SoftwareTools', 'VolunteeringWork', 'WorkshopsOrg', 'WorkshopsDomain',
       'AwardsNLPkey', 'ExternalCertificatesKey',
       'ExternalCertificatesDomain'],
      dtype='object')

In [20]:
#selection of dataset
data= data[['ProgLanguages','TierLevel']]
data['ProgLanguages']=data['ProgLanguages'].str.split('##')
data = data.dropna()

In [21]:
len(data)

34

In [22]:
data = data[data['TierLevel'] == 1.0] 

In [23]:
len(data)

27

In [25]:
#finding frequency of domains
domain_count={}
for v,k in data['ProgLanguages'].iteritems():
    for i in k:
        if i in domain_count:
            domain_count[i]=domain_count[i]+1
        else:
            domain_count[i]=1

In [27]:
#domains with frequency count
domain_count
#domain_count.pop('') 

{' html': 1,
 ' javascript': 1,
 'ajax': 2,
 'c': 25,
 'c\n': 2,
 'c++': 12,
 'c++(stl)': 1,
 'cryptography': 2,
 'css': 13,
 'django': 2,
 'html': 12,
 'html5': 3,
 'java': 15,
 'java ': 1,
 'javascript': 21,
 'lex': 2,
 'mysql': 1,
 'nodejs': 3,
 'opencv': 2,
 'php': 15,
 'python': 24,
 'r': 9,
 'r\n': 1,
 'reactjs': 2,
 'scala': 2,
 'w3.css': 1,
 'yacc\n': 2}

In [24]:
data.head()

,ProgLanguages,TierLevel
0,"[python, c++, c\n]",1.0
2,"[c, python, java, html, css, php, mysql]",1.0
4,"[python, java, c, javascript, html, css, php]",1.0
5,"[python, php, javascript, css, w3.css, html5, ...",1.0
7,"[c, python, java, html, css, javascript, php]",1.0


In [28]:
#finding support: frequency/total
domain_sup={}
for k, v in domain_count.items():
    domain_sup[k] = v/272

In [32]:
#calulate MIS for each itemset

#GIVE PARAMETERS

#Step 1: find SD
#Lambda(mean)
lamb = 0
for val in domain_count.values(): 
    lamb += val 
lamb = lamb / len(domain_count) 
#Consider alpha(between 1 and 0) 0.5
alpha= 0.1
ls=5

In [33]:

def mis(domain_count,ls,alpha):
    sd=lamb*(1-alpha)
    mis=[]
    misdict={}
    mis_val={}
    for k, v in domain_count.items():
        m=domain_count[k]-sd
        if(m<ls):
            mis.append(ls)
            misdict[domain_count[k]]=ls
            mis_val[k]=ls
        else:
            mis.append(m)
            misdict[domain_count[k]]=m
            mis_val[k]=m
    #print(mis_val)
    return mis_val

mis_val=mis(domain_count,ls,alpha)

In [34]:
mis_val

{' html': 5,
 ' javascript': 5,
 'ajax': 5,
 'c': 19.1,
 'c\n': 5,
 'c++': 6.1,
 'c++(stl)': 5,
 'cryptography': 5,
 'css': 7.1,
 'django': 5,
 'html': 6.1,
 'html5': 5,
 'java': 9.1,
 'java ': 5,
 'javascript': 15.1,
 'lex': 5,
 'mysql': 5,
 'nodejs': 5,
 'opencv': 5,
 'php': 9.1,
 'python': 18.1,
 'r': 5,
 'r\n': 5,
 'reactjs': 5,
 'scala': 5,
 'w3.css': 5,
 'yacc\n': 5}

In [41]:
mis_min2=min(mis_val.values())
L_2={}
for k, v in domain_count.items():
        if(domain_count[k]>=mis_val[k]):
            L_2[k]=mis_val[k]

#Step 2
#Sort L
temp= sorted(L_2.items(), key=operator.itemgetter(0))
L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)
#2 ITEMSETS 
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
n = 2
C2=findsubsets(L_21, n)

#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row['ProgLanguages'] and v[0] in row['ProgLanguages']):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L_12={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_12[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L_12.items(), key=operator.itemgetter(0))
L_12_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_12_sorted

[('python,c', 22),
 ('javascript,c', 20),
 ('python,javascript', 20),
 ('java,c', 15),
 ('python,php', 15),
 ('php,c', 14),
 ('php,javascript', 14),
 ('python,java', 14)]

In [43]:
#3 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 3
C3=findsubsets(L_21, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProgLanguages'] and b[0] in row['ProgLanguages'] and c[0] in row['ProgLanguages']):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L_3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L_3.items(), key=operator.itemgetter(0))
L_3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_3_sorted

[('python,javascript,c', 19),
 ('python,c,java', 14),
 ('python,php,c', 14),
 ('python,php,javascript', 14)]

In [45]:
#4 ITEMSETS
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n))) 
n = 4
C4=findsubsets(L_21, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row['ProgLanguages'] and b[0] in row['ProgLanguages'] and c[0] in row['ProgLanguages'] and d[0] in row['ProgLanguages']):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L_4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L_4.items(), key=operator.itemgetter(0))
L_4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
L_4_sorted

[]